In [12]:
import tkinter as tk
from tkinter import ttk, messagebox
import mysql.connector
from datetime import datetime

class HotelManagementApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Hotel Management System")
        self.master.geometry("800x600")

        self.notebook = ttk.Notebook(self.master)
        self.notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.available_rooms_frame = ttk.Frame(self.notebook)
        self.reservations_frame = ttk.Frame(self.notebook)
        self.check_in_frame = ttk.Frame(self.notebook)
        self.check_out_frame = ttk.Frame(self.notebook)

        self.notebook.add(self.available_rooms_frame, text="Available Rooms")
        self.notebook.add(self.reservations_frame, text="Reservations")
        self.notebook.add(self.check_in_frame, text="Check In")
        self.notebook.add(self.check_out_frame, text="Check Out")

        self.setup_available_rooms_tab()
        self.setup_reservations_tab()
        self.setup_check_in_tab()
        self.setup_check_out_tab()

    def setup_available_rooms_tab(self):
        ttk.Label(self.available_rooms_frame, text="Check-in Date (YYYY-MM-DD):").grid(row=0, column=0, padx=5, pady=5)
        self.check_in_entry = ttk.Entry(self.available_rooms_frame)
        self.check_in_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(self.available_rooms_frame, text="Check-out Date (YYYY-MM-DD):").grid(row=1, column=0, padx=5, pady=5)
        self.check_out_entry = ttk.Entry(self.available_rooms_frame)
        self.check_out_entry.grid(row=1, column=1, padx=5, pady=5)

        ttk.Button(self.available_rooms_frame, text="Check Availability", command=self.check_availability).grid(row=2, column=0, columnspan=2, pady=10)

        self.available_rooms_tree = ttk.Treeview(self.available_rooms_frame, columns=("Room Number", "Room Type", "Price"), show="headings")
        self.available_rooms_tree.heading("Room Number", text="Room Number")
        self.available_rooms_tree.heading("Room Type", text="Room Type")
        self.available_rooms_tree.heading("Price", text="Price")
        self.available_rooms_tree.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

    def setup_reservations_tab(self):
        self.reservations_tree = ttk.Treeview(self.reservations_frame, columns=("Reservation ID", "Customer Name", "Room Number", "Check-in", "Check-out", "Status"), show="headings")
        self.reservations_tree.heading("Reservation ID", text="Reservation ID")
        self.reservations_tree.heading("Customer Name", text="Customer Name")
        self.reservations_tree.heading("Room Number", text="Room Number")
        self.reservations_tree.heading("Check-in", text="Check-in")
        self.reservations_tree.heading("Check-out", text="Check-out")
        self.reservations_tree.heading("Status", text="Status")
        self.reservations_tree.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        ttk.Button(self.reservations_frame, text="Refresh Reservations", command=self.refresh_reservations).pack(pady=10)

    def setup_check_in_tab(self):
        ttk.Label(self.check_in_frame, text="Customer Name:").grid(row=0, column=0, padx=5, pady=5)
        self.customer_name_entry = ttk.Entry(self.check_in_frame)
        self.customer_name_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(self.check_in_frame, text="Phone:").grid(row=1, column=0, padx=5, pady=5)
        self.customer_phone_entry = ttk.Entry(self.check_in_frame)
        self.customer_phone_entry.grid(row=1, column=1, padx=5, pady=5)

        ttk.Label(self.check_in_frame, text="Email:").grid(row=2, column=0, padx=5, pady=5)
        self.customer_email_entry = ttk.Entry(self.check_in_frame)
        self.customer_email_entry.grid(row=2, column=1, padx=5, pady=5)

        ttk.Label(self.check_in_frame, text="Room Number:").grid(row=3, column=0, padx=5, pady=5)
        self.room_number_entry = ttk.Entry(self.check_in_frame)
        self.room_number_entry.grid(row=3, column=1, padx=5, pady=5)

        ttk.Label(self.check_in_frame, text="Check-in Date (YYYY-MM-DD):").grid(row=4, column=0, padx=5, pady=5)
        self.check_in_date_entry = ttk.Entry(self.check_in_frame)
        self.check_in_date_entry.grid(row=4, column=1, padx=5, pady=5)

        ttk.Label(self.check_in_frame, text="Check-out Date (YYYY-MM-DD):").grid(row=5, column=0, padx=5, pady=5)
        self.check_out_date_entry = ttk.Entry(self.check_in_frame)
        self.check_out_date_entry.grid(row=5, column=1, padx=5, pady=5)

        ttk.Button(self.check_in_frame, text="Check In", command=self.check_in_customer).grid(row=6, column=0, columnspan=2, pady=10)

    def setup_check_out_tab(self):
        ttk.Label(self.check_out_frame, text="Reservation ID:").grid(row=0, column=0, padx=5, pady=5)
        self.reservation_id_entry = ttk.Entry(self.check_out_frame)
        self.reservation_id_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Button(self.check_out_frame, text="Check Out", command=self.check_out_customer).grid(row=1, column=0, columnspan=2, pady=10)

    def connect_to_database(self):
        try:
            connection = mysql.connector.connect(
                host="localhost",
                user="root",
                password="",
                database="hotel"
            )
            return connection
        except mysql.connector.Error as error:
            messagebox.showerror("Database Error", f"Error connecting to the database: {error}")
            return None

    def check_availability(self):
        check_in_date = self.check_in_entry.get()
        check_out_date = self.check_out_entry.get()

        try:
            datetime.strptime(check_in_date, "%Y-%m-%d")
            datetime.strptime(check_out_date, "%Y-%m-%d")
        except ValueError:
            messagebox.showerror("Input Error", "Invalid date format. Please use YYYY-MM-DD.")
            return

        connection = self.connect_to_database()
        if not connection:
            return

        try:
            cursor = connection.cursor()
            query = """
            SELECT room_number, room_type, price 
            FROM room_information
            WHERE room_status = 'available' 
            AND room_number NOT IN (
                SELECT room_number 
                FROM reservation_information
                WHERE check_in_date <= %s AND check_out_date >= %s
            )
            """
            cursor.execute(query, (check_out_date, check_in_date))
            results = cursor.fetchall()

            self.available_rooms_tree.delete(*self.available_rooms_tree.get_children())
            for room in results:
                self.available_rooms_tree.insert("", "end", values=room)

        except mysql.connector.Error as error:
            messagebox.showerror("Query Error", f"Error executing query: {error}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

    def refresh_reservations(self):
        connection = self.connect_to_database()
        if not connection:
            return

        try:
            cursor = connection.cursor()
            query = """
            SELECT r.reservation_id, c.customer_name, r.room_number, 
                   r.check_in_date, r.check_out_date, r.reservation_status
            FROM reservation_information r
            INNER JOIN customer_information c ON r.customer_id = c.customer_id
            """
            cursor.execute(query)
            results = cursor.fetchall()

            self.reservations_tree.delete(*self.reservations_tree.get_children())
            for reservation in results:
                self.reservations_tree.insert("", "end", values=reservation)

        except mysql.connector.Error as error:
            messagebox.showerror("Query Error", f"Error executing query: {error}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

    def check_in_customer(self):
        customer_name = self.customer_name_entry.get()
        customer_phone = self.customer_phone_entry.get()
        customer_email = self.customer_email_entry.get()
        room_number = self.room_number_entry.get()
        check_in_date = self.check_in_date_entry.get()
        check_out_date = self.check_out_date_entry.get()

        if not all([customer_name, customer_phone, customer_email, room_number, check_in_date, check_out_date]):
            messagebox.showerror("Input Error", "All fields are required.")
            return

        connection = self.connect_to_database()
        if not connection:
            return

        try:
            cursor = connection.cursor()

            # Insert new customer
            insert_customer_query = """
            INSERT INTO customer_information (customer_name, customer_phone, customer_email)
            VALUES (%s, %s, %s)
            """
            cursor.execute(insert_customer_query, (customer_name, customer_phone, customer_email))
            customer_id = cursor.lastrowid

            # Create new reservation
            insert_reservation_query = """
            INSERT INTO reservation_information (customer_id, room_number, check_in_date, check_out_date, reservation_date, reservation_status)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_reservation_query, (customer_id, room_number, check_in_date, check_out_date, datetime.now(), 'confirmed'))

            # Update room status
            update_room_query = """
            UPDATE room_information
            SET room_status = 'occupied'
            WHERE room_number = %s
            """
            cursor.execute(update_room_query, (room_number,))

            connection.commit()
            messagebox.showinfo("Success", "Customer checked in successfully.")
            self.clear_check_in_fields()

        except mysql.connector.Error as error:
            messagebox.showerror("Database Error", f"Error checking in customer: {error}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

    def check_out_customer(self):
        reservation_id = self.reservation_id_entry.get()

        if not reservation_id:
            messagebox.showerror("Input Error", "Reservation ID is required.")
            return

        connection = self.connect_to_database()
        if not connection:
            return

        try:
            cursor = connection.cursor()

            # Get room number for the reservation
            get_room_query = """
            SELECT room_number
            FROM reservation_information
            WHERE reservation_id = %s
            """
            cursor.execute(get_room_query, (reservation_id,))
            result = cursor.fetchone()

            if not result:
                messagebox.showerror("Error", "Reservation not found.")
                return

            room_number = result[0]

            # Update reservation status
            update_reservation_query = """
            UPDATE reservation_information
            SET reservation_status = 'checked_out'
            WHERE reservation_id = %s
            """
            cursor.execute(update_reservation_query, (reservation_id,))

            # Update room status
            update_room_query = """
            UPDATE room_information
            SET room_status = 'available'
            WHERE room_number = %s
            """
            cursor.execute(update_room_query, (room_number,))

            connection.commit()
            messagebox.showinfo("Success", "Customer checked out successfully.")
            self.reservation_id_entry.delete(0, tk.END)

        except mysql.connector.Error as error:
            messagebox.showerror("Database Error", f"Error checking out customer: {error}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

    def clear_check_in_fields(self):
        self.customer_name_entry.delete(0, tk.END)
        self.customer_phone_entry.delete(0, tk.END)
        self.customer_email_entry.delete(0, tk.END)
        self.room_number_entry.delete(0, tk.END)
        self.check_in_date_entry.delete(0, tk.END)
        self.check_out_date_entry.delete(0, tk.END)

def main():
    root = tk.Tk()
    app = HotelManagementApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

In [1]:
pip install mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/5f/c3/b4c36fbc65c119fca83f258557573d8cd296da5e198d3752c669f8e871e2/mysql_connector_python-9.0.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB 262.6 kB/s eta 0:00:55
   ---------------------------------------- 0.1/14.3 MB 363.1 kB/s eta 0:00:40
   ---------------------------------------- 0.2/14.3 MB 756.6 kB/s eta 0:00:19
    --------------------------------------- 0.4/14.3 MB 1.5 MB/s eta 0:00:10
   -- ------------------------------------- 0.7/14.3 MB 2.6 MB/s eta 0:00:06
   --- ------------------------------------ 1.3/14.3 MB 3.8 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/14.3 MB 5.5 MB/s eta 0:00:03
   ----


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
